In [4]:
import pandas as pd
import utils
import plotly.graph_objects as go
import instrument

In [5]:
pair = "EUR_USD"
granularity = "H1"
ma_list = [16,64]
i_pair = instrument.Instrument.get_instrument_by_name("EUR_USD")

In [6]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))
non_cols = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [7]:
df_ma = df[['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c']].copy()
for ma in ma_list:  
    df_ma[f'MA_{ma}'] = df_ma.mid_c.rolling(window=ma).mean()
df_ma.dropna(inplace=True)
df_ma.reset_index(drop=True, inplace=True)

In [8]:
df_ma.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64
0,2023-07-18T16:00:00.000000000Z,1.12224,1.12286,1.12092,1.12147,1.124358,1.123503
1,2023-07-18T17:00:00.000000000Z,1.12146,1.12212,1.12092,1.12188,1.124191,1.123480
2,2023-07-18T18:00:00.000000000Z,1.12188,1.12312,1.12138,1.12290,1.124073,1.123470
3,2023-07-18T19:00:00.000000000Z,1.12291,1.12308,1.12244,1.12303,1.123925,1.123453
4,2023-07-18T20:00:00.000000000Z,1.12304,1.12318,1.12271,1.12284,1.123779,1.123446


In [9]:
def is_trade(row):
    if row.DIFF >= 0 and row.DIFF_PREV < 0:
        return True
    if row.DIFF <= 0 and row.DIFF_PREV > 0:
        return True
    return False

In [10]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64
df_ma['DIFF_PREV'] = df_ma.DIFF.shift(1)
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1)
df_trades = df_ma[df_ma.IS_TRADE==True].copy()

In [15]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
7,2023-07-18T23:00:00.000000000Z,1.12316,1.12322,1.12252,1.12284,1.123338,1.123506,-0.000167,0.000002,True
147,2023-07-26T19:00:00.000000000Z,1.11040,1.11070,1.10784,1.10964,1.107142,1.106852,0.000290,-0.000048,True
171,2023-07-27T19:00:00.000000000Z,1.09738,1.09743,1.09660,1.09672,1.105968,1.106365,-0.000396,0.000261,True
221,2023-07-31T21:00:00.000000000Z,1.09958,1.09994,1.09936,1.09986,1.101707,1.101669,0.000039,-0.000003,True
227,2023-08-01T03:00:00.000000000Z,1.09853,1.09892,1.09840,1.09857,1.100351,1.100358,-0.000006,0.000012,True


In [16]:
df_ma

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
0,2023-07-18T16:00:00.000000000Z,1.12224,1.12286,1.12092,1.12147,1.124358,1.123503,0.000855,NaN,False
1,2023-07-18T17:00:00.000000000Z,1.12146,1.12212,1.12092,1.12188,1.124191,1.123480,0.000710,0.000855,False
2,2023-07-18T18:00:00.000000000Z,1.12188,1.12312,1.12138,1.12290,1.124073,1.123470,0.000603,0.000710,False
3,2023-07-18T19:00:00.000000000Z,1.12291,1.12308,1.12244,1.12303,1.123925,1.123453,0.000472,0.000603,False
4,2023-07-18T20:00:00.000000000Z,1.12304,1.12318,1.12271,1.12284,1.123779,1.123446,0.000333,0.000472,False
...,...,...,...,...,...,...,...,...,...,...
3931,2024-03-06T12:00:00.000000000Z,1.08758,1.08788,1.08674,1.08688,1.086043,1.085346,0.000697,0.000677,False
3932,2024-03-06T13:00:00.000000000Z,1.08690,1.08851,1.08685,1.08850,1.086219,1.085416,0.000802,0.000697,False
3933,2024-03-06T14:00:00.000000000Z,1.08848,1.08895,1.08830,1.08870,1.086407,1.085488,0.000918,0.000802,False
3934,2024-03-06T15:00:00.000000000Z,1.08850,1.09089,1.08814,1.09051,1.086717,1.085587,0.001130,0.000918,False


In [12]:
df_plot = df_ma.iloc[0:24].copy()

In [13]:
df_plot.shape

(24, 10)

In [14]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
for ma in ma_list:  
    col = f"MA_{ma}"
    fig.add_trace(go.Scatter(x=df_plot.time, 
        y=df_plot[col],
        line=dict(width=2),
        line_shape='spline',
        name=col
        ))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()